## This notebook does slew simulations, check all aos components (M1M3, M2, hexapods) behavior during the slew-and-track process, and applies MTAOS corrections for each visit.

This is expected to work both for SUMMIT and NCSA

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
from lsst.ts import salobj
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import logging

from lsst.ts.idl.enums import MTHexapod

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from aosTools import *

plt.jet();

In [ ]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
from lsst_efd_client import EfdClient

if summit:
    client = EfdClient('summit_efd')
else:
    client = EfdClient('ncsa_teststand_efd')
csc_index = 1

In [ ]:
#index is an integter which helps avoid multple users starting same controller
script = salobj.Controller("Script", index=8)

In [ ]:
mtcs = MTCS(script.domain)
#mtcs.set_rem_loglevel(40)
mtcs.set_rem_loglevel(logging.CRITICAL) 

In [ ]:
mtcs.set_rem_loglevel(logging.CRITICAL)


In [ ]:
await mtcs.start_task

In [ ]:
mtcs.components_attr

In [ ]:
ptg = mtcs.rem.mtptg
mount = mtcs.rem.mtmount
rot = mtcs.rem.mtrotator
camhex = mtcs.rem.mthexapod_1
m2hex = mtcs.rem.mthexapod_2
m1m3 = mtcs.rem.mtm1m3
m2 = mtcs.rem.mtm2
aos = mtcs.rem.mtaos

### Get Ptg ready

In [ ]:
await ptg.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await ptg.cmd_start.set_start()

In [ ]:
await salobj.set_summary_state(ptg, salobj.State.ENABLED)

### Get Mount ready

In [ ]:
await mount.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
sim_evt = await mount.evt_simulationMode.aget(timeout=5)
print('simulation mode? ', sim_evt.mode, pd.to_datetime(sim_evt.private_sndStamp, unit='s'))

In [ ]:
a = await mount.evt_softwareVersions.aget(timeout=5)
print('software version? ', a.cscVersion, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
a = await mount.evt_cameraCableWrapFollowing.aget()
print(a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)

In [ ]:
await mount.cmd_start.set_start()

In [ ]:
await salobj.set_summary_state(mount, salobj.State.ENABLED)

### Get Rotator ready

In [ ]:
await checkSlewCompStates(ptg, mount, rot)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
sim_evt = await rot.evt_simulationMode.aget(timeout=5)
print('simulation mode? ', sim_evt.mode, pd.to_datetime(sim_evt.private_sndStamp, unit='s'))

In [ ]:
a = await mount.tel_elevation.next(flush=True, timeout=5)
print("mount elevation Angle = ", a.actualPosition)
a = await mount.tel_azimuth.next(flush=True, timeout=5)
print("mount azimuth angle = ", a.actualPosition)
a = await mount.tel_cameraCableWrap.next(flush=True, timeout=5)
print("CCW angle = ", a.actualPosition, " Needs to be within 2.2 deg of rotator angle ")
b = await rot.tel_rotation.next(flush=True, timeout=5)
print("rot angle = ", b.actualPosition, "   diff = ", (b.actualPosition - a.actualPosition))

In [ ]:
a = await rot.evt_errorCode.aget()
print(a.errorReport, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
a = await mount.evt_cameraCableWrapFollowing.aget(timeout=5.)
print(a.enabled)

If the Camera Cable Warp is not following:
Transition the MTMount to StandbyState, wait 10 seconds, transition to EnableState

In [ ]:
await mount.cmd_enableCameraCableWrapFollowing.set_start()

In [ ]:
await rot.cmd_clearError.set_start()

In [ ]:
await rot.cmd_start.set_start()

In [ ]:
await salobj.set_summary_state(mount, salobj.State.DISABLED)
await salobj.set_summary_state(mount, salobj.State.ENABLED)
a = await mount.evt_cameraCableWrapFollowing.aget()
print('CCW folowing? ', a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
await rot.cmd_move.set_start(0)

### Get MTAOS Ready

In [ ]:
await aos.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.ENABLED) #, settingsToApply='default') #leave this out!!
#two levels of defaults: csc defaults, config repo; right now csc defaults, they will move to config repo; 
#but to load defaults from config repo, we do not need to supply settingsToApply

In [ ]:
a = await aos.evt_softwareVersions.aget(timeout=5)
print('software version? ', a.cscVersion, pd.to_datetime(a.private_sndStamp, unit='s'))

#this is MTAOS version. How do I get OFC version? and WEP version?

In [ ]:
def printLogMessage(data):
    print(f"{data.level}: {data.message}")
aos.evt_logMessage.callback = printLogMessage
await aos.cmd_setLogLevel.set_start(level=10, timeout=5)

### Check that all components are ready

In [ ]:
await checkAOSCompStates(m1m3, m2, camhex, m2hex)

In [ ]:
await checkSlewCompStates(ptg, mount, rot)

In [ ]:
await checkAOSSummaryStates(aos, m1m3,m2,camhex,m2hex)

### Get M1M3 ready (Mount telemetry mode)

In [ ]:
#Move to zenith (so that we can start m1m3 with LUT in mount telemetry mode)
await mount.cmd_moveToTarget.set_start(azimuth=0, elevation=90)

In [ ]:
await m1m3.evt_heartbeat.next(flush=True, timeout=5)

Check that the M1M3 is in mount telemetry mode.  For this you have to login to m1m3-crio-ss.cp.lsst.org!!

In [ ]:
await salobj.set_summary_state(m1m3, salobj.State.STANDBY)

In [ ]:
print('Re-enabling M1M3')
#await salobj.set_summary_state(m1m3, salobj.State.STANDBY) 
await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM1M3(m1m3)

In [ ]:
await m1m3.cmd_abortRaiseM1M3.set_start(timeout=15.)

In [ ]:
await checkM1M3Error(m1m3)

In [ ]:
await lowerM1M3(m1m3)

In [ ]:
m1m3.

### Get M2 ready

In [ ]:
await m2.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await checkAOSCompStates(m1m3,m2,camhex,m2hex)

In [ ]:
await salobj.set_summary_state(m2, salobj.State.OFFLINE) 

In [ ]:
await salobj.set_summary_state(m2, salobj.State.STANDBY) 

In [ ]:
await salobj.set_summary_state(m2, salobj.State.DISABLED) 

In [ ]:
#remember to reset interlock
await salobj.set_summary_state(m2, salobj.State.ENABLED) 

In [ ]:
a = await m2.evt_errorCode.aget()
print(a.errorCode, a.errorReport, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
await m2.cmd_enterControl.set_start(timeout=.5)

In [ ]:
await m2.cmd_exitControl.set_start(timeout=.5)

In [ ]:
await m2.cmd_enable.set_start(timeout=5.)

In [ ]:
await m2.cmd_disable.set_start(timeout=5.)

In [ ]:
await m2.cmd_clearErrors.set_start(timeout=15.)

In [ ]:
await readyM2(m2)

### Get camHex ready

In [ ]:
target_evt = await mount.evt_target.aget(timeout=5.)
print("Mount target elevation = ", target_evt.elevation, "  @  ", pd.to_datetime(target_evt.private_sndStamp, unit='s'))
print("Mount target azimuth   = ", target_evt.azimuth, "  @  ", pd.to_datetime(target_evt.private_sndStamp, unit='s'))
target_evt = await rot.evt_target.aget(timeout=5.)
print("Rotator target position = ", target_evt.position, "  @  ", pd.to_datetime(target_evt.private_sndStamp, unit='s'))

In [ ]:
#if any of the above doesn't work
#await mount.cmd_moveToTarget.set_start(azimuth=0, elevation=81)
await rot.cmd_move.set_start(position=0)

In [ ]:
await camhex.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
sim_evt = await camhex.evt_simulationMode.aget(timeout=5)
print('simulation mode? ', sim_evt.mode, pd.to_datetime(sim_evt.private_sndStamp, unit='s'))

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.STANDBY)

In [ ]:
await camhex.cmd_clearError.set_start()

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.DISABLED, settingsToApply='default')

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.ENABLED, settingsToApply='default')

Move the Pivot point by 100um to make sure that the configuration event gets published

In [ ]:
await camhex.cmd_setPivot.set_start(x=0,y=0,z=-2758300)

In [ ]:
await camhex.cmd_setPivot.set_start(x=0,y=0,z=-2758400)

In [ ]:
await readyHexaForAOS(camhex)

In [ ]:
await printHexaUncompensatedAndCompensated(m2hex)

In [ ]:
await printHexaUncompensatedAndCompensated(camhex)

### Get m2Hex ready

In [ ]:
await salobj.set_summary_state(m2hex, salobj.State.ENABLED, settingsToApply='default') #works if it is offline

In [ ]:
sim_evt = await m2hex.evt_simulationMode.aget(timeout=5)
print('simulation mode? ', sim_evt.mode, pd.to_datetime(sim_evt.private_sndStamp, unit='s'))

In [ ]:
await readyHexaForAOS(m2hex)

In [ ]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)
df = await client.select_time_series('lsst.sal.ESS.temperature8Ch', '*', start, end, csc_index)
if len(df)>0:
    fig, ax = plt.subplots(1,1, figsize=(15,4))
    for i in range(1,7):
        plt.plot(getattr(df, 'temperatureC%02d'%i))
    plt.grid()
else:
    print('No temperature data on the camera hexapod in the past 1000s.')

### Do a few slews

In [ ]:
await m1m1.error

#### Do 4 slews, then stop tracking

In [ ]:
#comment and uncomment the below depending on which components are expected to be on
#by default, MTCS checks on every component (during the slew).
mtcs.check.mtaos = False
mtcs.check.mtm1m3 = True
mtcs.check.mtm2 = True
mtcs.check.mthexapod_1 = True
mtcs.check.mthexapod_2 = True
mtcs.check.mtdome = False
mtcs.check.mtdometrajectory = False

In [ ]:
def printLogMessage(data):
    print(f"{data.level}: {data.message}")
aos.evt_logMessage.callback = printLogMessage
await aos.cmd_setLogLevel.set_start(level=10, timeout=5)

In [ ]:
dec = -34.
for j in range(2):
    for i in range(2):
        
        a = await mount.evt_cameraCableWrapFollowing.aget()
        print('CCW folowing? ', a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))
        if not a.enabled:
            break
        time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
        ra = time_and_date.lst + 0.5 - 3.5/15.0 * i
        
        aa = await mount.tel_elevation.next(flush=True, timeout=5.)
        current_el = aa.actualPosition
        
        obs_time = salobj.astropy_time_from_tai_unix(salobj.current_tai() + 0.) #with 0s delay
        azel = mtcs.azel_from_radec(ra=ra, dec=dec, time=obs_time)
        target_el = azel.alt.value
        
        while abs(target_el - current_el)>0.3:
            print('moving from elevation %.1f deg to %.1f deg'%(current_el, target_el), Time.now())
            await moveMountConstantV(mount, current_el, target_el)
        
            aa = await mount.tel_elevation.next(flush=True, timeout=5.)
            current_el = aa.actualPosition
        
            time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
            ra = time_and_date.lst + 0.5 - 3.5/15.0 * i
    
            obs_time = salobj.astropy_time_from_tai_unix(salobj.current_tai() + 0.) #with 0s delay
            azel = mtcs.azel_from_radec(ra=ra, dec=dec, time=obs_time)
            target_el = azel.alt.value
        
        await moveRotTo0(rot, 10.0)
        
        print('start a slew, elevation diff = ', abs(target_el - current_el), Time.now())
        a = await mount.evt_cameraCableWrapFollowing.aget()
        print('CCW folowing? ', a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))
        if not a.enabled:
            break
            
        await aos.cmd_resetCorrection.set_start()
        await aos.cmd_issueCorrection.set_start() 
        k = j*2+i
        zernikes = np.zeros(19)
        if k==0:
            zernikes[4-4] = 1 #add 1um of z4
        elif k==1:
            zernikes[5-4] = 1 #add 1um of z5
        elif k==2:
            zernikes[7-4] = 1 #add 1um of z7
        elif k==3:
            zernikes[9-4] = 1 #add 1um of z9
        await aos.cmd_addAberration.set_start(wf = zernikes)
        await aos.cmd_issueCorrection.set_start() 
        await asyncio.sleep(2.0)

        ofc_dict =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

        rotAngle = 10.0
        print('zero-indexed: ', k, 'rot = ', rotAngle)
        #b = await mount.tel_cameraCableWrap.next(flush=True, timeout=5) #CCW doesn't follow rotator!!!
        # await mtcs.slew_icrs(ra=ra, dec=dec, rot=b.actualPosition, rot_type=RotType.PhysicalSky)
        await mtcs.slew_icrs(ra=ra, dec=dec, rot=rotAngle, rot_type=RotType.PhysicalSky)
        #await mtcs.slew_icrs(ra=ra, dec=dec, rot=0.0, rot_type=RotType.Physical)
        await asyncio.sleep(39.)
        await mtcs.stop_tracking()

In [ ]:
await mtcs.stop_tracking()

#### Plot the above process

In [ ]:
#end = Time(datetime.now(), scale='tai')
end = Time('2021-07-13T19:18:00', scale='tai')
start = end - timedelta(seconds=1000)

In [ ]:
dfm2

In [ ]:
dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedElevationForces', '*', start, end, csc_index)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.application', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot((dfm.actualPosition-dfm.actualPosition[0])*100, '--', label='mount elevation x 100 (deg)')
plt.plot(dfm1m3.yForces0-dfm1m3.yForces0[0], label='M1M3 elevation y-force 101 (N)')
plt.plot((dfm2.lutGravity0-dfm2.lutGravity0[0])*10, label='M2 elevation force B1 x 10 (N)')
plt.plot(dfh1.position1-dfh1.position1[0], label='Camera hexapod y (um)')
plt.plot(dfh2.position1-dfh2.position1[0], label='M2 hexapod y (um)')
plt.grid()
plt.legend()
plt.title('Changes in telemetry')

#### Check if the telescope is in tracking mode. If yes, need to stop stacking.

The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [ ]:
await mtcs.stop_tracking()

### put mount elevation back to 90 deg, so that we can lower M1M3

In [ ]:
aa = await mount.tel_elevation.next(flush=True, timeout=5.)
current_el = aa.actualPosition

In [ ]:
await moveMountConstantV(mount,current_el, 90)

In [ ]:
await lowerM1M3(m1m3)

#### Close up. Put all simulators to standby

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await salobj.set_summary_state(aos, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(ptg, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.STANDBY)

In [ ]:
await camhex.cmd_disable.set_start()

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.DISABLED)

In [ ]:
await salobj.set_summary_state(m2hex, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(m2, salobj.State.DISABLED)

In [ ]:
await salobj.set_summary_state(m2, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(m2, salobj.State.OFFLINE)

In [ ]:
await lowerM1M3(m1m3)

In [ ]:
await salobj.set_summary_state(m1m3, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)

In [ ]:
await rot.cmd_clearError.set_start()

In [ ]:
await m2.cmd_exitControl.set_start()

In [ ]:
await checkAOSCompStates(m1m3,m2,camhex, m2hex)

In [ ]:
await checkSlewCompStates(ptg,mount, rot)